In [ ]:
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
data_path = "..."

In [6]:
MEMBERS = f"{data_path}/members.csv"
SONG_FEAT = f"{data_path}/song_extra_info.csv"
SONGS = f"{data_path}/songs.csv"
TRAIN = f"{data_path}/train.csv"
MERGE = f"{data_path}/train_data.csv"

In [7]:
members_df = pd.read_csv(MEMBERS)
song_feat_df = pd.read_csv(SONG_FEAT)
songs_df = pd.read_csv(SONGS)
train_df = pd.read_csv(TRAIN)

In [8]:
train_df = train_df.groupby("msno").sample(frac=0.1, random_state=42)

In [9]:
le_msno = LabelEncoder()
le_song_id = LabelEncoder()

le_msno.fit(train_df["msno"])
le_song_id.fit(train_df["song_id"])

train = pd.DataFrame({"msno": le_msno.transform(train_df["msno"].values), "song_id": le_song_id.transform(train_df["song_id"].values), "target": train_df["target"]})

In [10]:
private_data = train.groupby("msno").sample(frac=0.3, random_state=42)
global_data = train.drop(private_data.index).sort_values(by="msno")

In [11]:
private_data.shape, global_data.shape

((221417, 3), (515803, 3))

In [12]:
# --- Конфигурация федерации ---
NUM_CLIENTS = 5
LOCAL_EPOCHS = 3
AGGREGATION_ROUNDS = 10
FACTORS = 50
REGULARIZATION = 0.01
TOP_N_RECOMMENDATIONS = 20  # число рекомендаций для генерации новых данных

In [14]:
# 1. Инициализация глобальной модели
global_model = AlternatingLeastSquares(factors=FACTORS,
                                      regularization=REGULARIZATION,
                                      use_gpu=False)

user_item = csr_matrix(
            (global_data['target'],
            (global_data['msno'], global_data['song_id']))
        )

global_model.fit(user_item)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
train_data, test_data = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

test_users = test_data["msno"]
test_items = test_data["song_id"]

user_factors = global_model.user_factors[test_users]
song_factors = global_model.item_factors[test_items]
predicted_scores = np.sum(user_factors * song_factors, axis=1)

auc = roc_auc_score(test_data["target"], predicted_scores)
precision = precision_score(test_data['target'], predicted_scores>0.6)
recall = recall_score(test_data['target'], predicted_scores>0.6)
print("AUC: ", auc)
print("precision: ", precision)
print("recall: ", recall)

AUC:  0.7497253242534933
precision:  0.9813635291342298
recall:  0.056006570001480946


In [18]:
# Предполагаем, что модель уже обучена и загружена
# with open('als_federated_model.pkl', 'rb') as f:
#     global_model = pickle.load(f)

# 2. Клиентская часть: локальное обучение с учётом новых рекомендаций
class Client:
    def __init__(self, client_id, base_data):
        self.id = client_id
        # Базовые реальные взаимодействия (user_id, item_id, weight)
        self.base_data = base_data
        # Локальная модель ALS
        self.model = AlternatingLeastSquares(factors=FACTORS,
                                            regularization=REGULARIZATION,
                                            use_gpu=False, iterations=5)

    def local_update(self, global_user_factors, global_item_factors):
        self.model.user_factors = global_user_factors.copy()
        self.model.item_factors = global_item_factors.copy()

        # Создаем CSR матрицу для пользователя
        user_items = csr_matrix(
            (self.base_data['target'],
             (np.zeros(len(self.base_data)),  # Все строки относятся к текущему пользователю
              self.base_data['song_id'])),
            shape=(1, global_item_factors.shape[0])
        )

        self.model.partial_fit_users([self.id], user_items)

        return self.model.user_factors, self.model.item_factors

In [ ]:
clients = []
for cid in tqdm(private_data["msno"].unique()):
    base_data = (private_data[private_data["msno"] == cid]).copy()
    clients.append(Client(cid, base_data))


100%|██████████| 25371/25371 [00:14<00:00, 1702.34it/s]


In [20]:
def aggregate(models_user, models_item):
    avg_user = np.mean(models_user, axis=0)
    avg_item = np.mean(models_item, axis=0)
    return avg_user, avg_item

for round_idx in tqdm(range(AGGREGATION_ROUNDS)):
    collected_user = []
    collected_item = []
    for client in clients[:10]:
        u_f, i_f = client.local_update(global_model.user_factors, global_model.item_factors)
        collected_user.append(u_f)
        collected_item.append(i_f)
    # Аггрегация
    mean_user, mean_item = aggregate(collected_user, collected_item)
    global_model.user_factors = mean_user
    global_model.item_factors = mean_item
    print(f"Round {round_idx+1}/{AGGREGATION_ROUNDS} aggregated with synthetic data")

 10%|█         | 1/10 [00:00<00:08,  1.10it/s]

Round 1/10 aggregated with synthetic data


 20%|██        | 2/10 [00:01<00:04,  1.85it/s]

Round 2/10 aggregated with synthetic data


 30%|███       | 3/10 [00:01<00:03,  2.27it/s]

...

Round 8/10 aggregated with synthetic data


 90%|█████████ | 9/10 [00:02<00:00,  3.86it/s]

Round 9/10 aggregated with synthetic data


100%|██████████| 10/10 [00:03<00:00,  3.09it/s]

Round 10/10 aggregated with synthetic data


In [25]:
test_users = test_data["msno"]
test_items = test_data["song_id"]

user_factors = global_model.user_factors[test_users]
song_factors = global_model.item_factors[test_items]
predicted_scores = np.sum(user_factors * song_factors, axis=1)

auc = roc_auc_score(test_data["target"], predicted_scores)
precision = precision_score(test_data['target'], predicted_scores>0.6)
recall = recall_score(test_data['target'], predicted_scores>0.6)
print("AUC: ", auc)
print("precision: ", precision)
print("recall: ", recall)

AUC:  0.7497168343396904
precision:  0.9813547321217843
recall:  0.05597964376590331


In [26]:
with open('als_federated_c_model.pkl', 'wb') as f:
    pickle.dump(global_model, f)